<a href="https://colab.research.google.com/github/hondore16/hondore16/blob/main/Pn1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import itertools
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# --- 1. ПРОВЕРКА GPU ---
print("Проверка доступности GPU...")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ GPU найден: {gpus[0]}")
else:
    print("⚠️ GPU не найден. Убедитесь, что включили его в 'Среда выполнения' -> 'Сменить среду выполнения'.")

# --- 2. ГЕНЕРАЦИЯ ДАННЫХ ---
def check_winner(board):
    grid = np.array(board).reshape(3, 3)
    win_x = 0
    win_o = 0

    # Проверка строк, столбцов и диагоналей
    lines = []
    for i in range(3):
        lines.append(grid[i, :])
        lines.append(grid[:, i])
    lines.append(grid.diagonal())
    lines.append(np.fliplr(grid).diagonal())

    for line in lines:
        if np.all(line == 1): win_x = 1
        if np.all(line == 2): win_o = 1
    return np.array([win_x, win_o])

print("\nГенерация датасета (19683 комбинации)...")
all_states = list(itertools.product([0, 1, 2], repeat=9))

X_data = []
Y_data = []

for state in all_states:
    # Вход: 18 бит (9 бит карта X, 9 бит карта O)
    layer_x = [1 if cell == 1 else 0 for cell in state]
    layer_o = [1 if cell == 2 else 0 for cell in state]
    input_vector = layer_x + layer_o

    winner = check_winner(state)

    X_data.append(input_vector)
    Y_data.append(winner)

X_data = np.array(X_data)
Y_data = np.array(Y_data)

# Перемешиваем данные для лучшего обучения
indices = np.arange(X_data.shape[0])
np.random.shuffle(indices)
X_data = X_data[indices]
Y_data = Y_data[indices]

print(f"Датасет готов. Размер: {X_data.shape}")

# --- 3. СОЗДАНИЕ МОЩНОЙ МОДЕЛИ ---
model = Sequential()

# Слой 1: Увеличили до 64 нейронов (было 16)
model.add(Dense(64, input_dim=18, activation='relu', name='hidden_layer_1'))

# Слой 2: Добавили еще один слой для лучшей абстракции
model.add(Dense(32, activation='relu', name='hidden_layer_2'))

# Выход: 2 нейрона (Победа X, Победа O)
model.add(Dense(2, activation='sigmoid', name='output_layer'))

# Используем Adam с чуть меньшим шагом обучения для стабильности
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

model.summary()

# --- 4. ОБУЧЕНИЕ ---
print("\nНачинаем обучение на GPU...")
# Увеличили количество эпох до 100
history = model.fit(X_data, Y_data, epochs=100, batch_size=64, verbose=1, validation_split=0.1)

# Итоговая проверка
loss, acc = model.evaluate(X_data, Y_data, verbose=0)
print(f"\n🏆 Итоговая точность: {acc*100:.2f}%")

# --- 5. ЭКСПОРТ ВЕСОВ (FIXED POINT) ---
# Теперь у нас 3 набора весов (Вход->Скрытый1, Скрытый1->Скрытый2, Скрытый2->Выход)
# Но для упрощения RTL (чтобы не писать три слоя на Verilog),
# я предлагаю оставить экспорт только для 1-го скрытого слоя и выхода,
# если точность вас устроит.
# Однако, раз мы добавили слой, нужно экспортировать все три.

SCALE = 128

def to_fixed(weights):
    return [int(round(w * SCALE)) for w in weights.flatten()]

print("\n--- КОД ДЛЯ ВСТАВКИ В VERILOG (Копируй в weights.txt) ---")

# Слой 1
w1, b1 = model.get_layer('hidden_layer_1').get_weights()
print(f"// Layer 1 Weights ({len(to_fixed(w1))} values)")
print(f"localparam [15:0] L1_W [0:{len(to_fixed(w1))-1}] = '{{ {', '.join(map(str, to_fixed(w1)))} }};")
print(f"// Layer 1 Biases")
print(f"localparam [15:0] L1_B [0:{len(to_fixed(b1))-1}] = '{{ {', '.join(map(str, to_fixed(b1)))} }};")

# Слой 2
w2, b2 = model.get_layer('hidden_layer_2').get_weights()
print(f"// Layer 2 Weights")
print(f"localparam [15:0] L2_W [0:{len(to_fixed(w2))-1}] = '{{ {', '.join(map(str, to_fixed(w2)))} }};")
print(f"// Layer 2 Biases")
print(f"localparam [15:0] L2_B [0:{len(to_fixed(b2))-1}] = '{{ {', '.join(map(str, to_fixed(b2)))} }};")

# Выходной слой
w3, b3 = model.get_layer('output_layer').get_weights()
print(f"// Output Layer Weights")
print(f"localparam [15:0] L3_W [0:{len(to_fixed(w3))-1}] = '{{ {', '.join(map(str, to_fixed(w3)))} }};")
print(f"// Output Layer Biases")
print(f"localparam [15:0] L3_B [0:{len(to_fixed(b3))-1}] = '{{ {', '.join(map(str, to_fixed(b3)))} }};")

Проверка доступности GPU...
✅ GPU найден: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')

Генерация датасета (19683 комбинации)...
Датасет готов. Размер: (19683, 18)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ hidden_layer_1 (Dense)          │ (None, 64)             │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_layer_2 (Dense)          │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,362 (13.13 KB)

 Trainable params: 3,362 (13.13 KB)

 Non-trainable params: 0 (0.00 B)


Начинаем обучение на GPU...
Epoch 1/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.6974 - loss: 0.5048 - val_accuracy: 0.6602 - val_loss: 0.3062
Epoch 2/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6824 - loss: 0.2581 - val_accuracy: 0.6714 - val_loss: 0.1926
Epoch 3/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6761 - loss: 0.1794 - val_accuracy: 0.6805 - val_loss: 0.1547
Epoch 4/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6744 - loss: 0.1477 - val_accuracy: 0.6851 - val_loss: 0.1270
Epoch 5/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6696 - loss: 0.1210 - val_accuracy: 0.6709 - val_loss: 0.1056
Epoch 6/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6766 - loss: 0.0966 - val_accuracy: 0.6541 - val_loss: 0.0890
Epoch 7/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6697 - loss: 0.0776 - val_accuracy: 0.6755 - val_loss: 0.0723
Epoch 8/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.

In [ ]:
import numpy as np
import tensorflow as tf
import itertools
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# --- 1. НАСТРОЙКА GPU ---
print("Проверка GPU...")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ GPU активен! ({gpus[0]})")
else:
    print("⚠️ GPU не найден. Обучение будет медленнее, но сработает.")

# --- 2. ГЕНЕРАЦИЯ И ЧИСТКА ДАННЫХ ---
def check_winner(board):
    grid = np.array(board).reshape(3, 3)
    win_x = 0
    win_o = 0

    lines = []
    for i in range(3):
        lines.append(grid[i, :])       # Строки
        lines.append(grid[:, i])       # Столбцы
    lines.append(grid.diagonal())      # Диагональ 1
    lines.append(np.fliplr(grid).diagonal()) # Диагональ 2

    for line in lines:
        if np.all(line == 1): win_x = 1
        if np.all(line == 2): win_o = 1
    return win_x, win_o

print("\nГенерация всех состояний (19683)...")
all_states = list(itertools.product([0, 1, 2], repeat=9))

X_raw = []
Y_raw = []

# Счетчики для статистики
count_x_wins = 0
count_o_wins = 0
count_draws = 0

for state in all_states:
    wx, wo = check_winner(state)

    # --- ФИЛЬТРАЦИЯ ---
    # Если победили оба одновременно - это мусорные данные, они путают сеть.
    # В реальной игре такого быть не может. Удаляем их.
    if wx == 1 and wo == 1:
        continue

    # Формируем вход (18 бит)
    layer_x = [1 if cell == 1 else 0 for cell in state]
    layer_o = [1 if cell == 2 else 0 for cell in state]

    X_raw.append(layer_x + layer_o)
    Y_raw.append([wx, wo])

    if wx: count_x_wins += 1
    elif wo: count_o_wins += 1
    else: count_draws += 1

X_data = np.array(X_raw)
Y_data = np.array(Y_raw)

print(f"Чистые данные готовы: {len(X_data)} примеров.")
print(f"Статистика: Побед X: {count_x_wins}, Побед O: {count_o_wins}, Ничьих/Пусто: {count_draws}")

# ПЕРЕМЕШИВАНИЕ (Критически важно!)
indices = np.arange(len(X_data))
np.random.shuffle(indices)
X_data = X_data[indices]
Y_data = Y_data[indices]

# --- 3. АРХИТЕКТУРА "ТЯЖЕЛОВЕС" ---
model = Sequential()

# Слой 1: 128 нейронов (чтобы запомнить все паттерны наверняка)
model.add(Dense(128, input_dim=18, activation='relu', name='layer1'))

# Слой 2: 64 нейрона (для обобщения)
model.add(Dense(64, activation='relu', name='layer2'))

# Выход: 2 нейрона
model.add(Dense(2, activation='sigmoid', name='output'))

# Learning rate 0.001 - золотая середина
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# --- 4. ОБУЧЕНИЕ ---
print("\nЗапуск обучения (Epochs=150)...")
# class_weight помогает обратить внимание на победы, которых меньше, чем ничьих
# Но с такой большой сетью она справится и так.
history = model.fit(
    X_data, Y_data,
    epochs=150,          # Даем больше времени
    batch_size=128,      # Пакеты побольше для стабильности градиента
    verbose=1,
    validation_split=0.1 # 10% данных оставим для честной проверки
)

# Итоговая проверка
loss, acc = model.evaluate(X_data, Y_data, verbose=0)
print(f"\n🔥🔥 ИТОГОВАЯ ТОЧНОСТЬ: {acc*100:.2f}% 🔥🔥")

if acc < 0.98:
    print("Странно... Попробуйте запустить еще раз, иногда неудачная инициализация весов мешает.")
else:
    print("Отличный результат! Можно генерировать веса.")

# --- 5. ЭКСПОРТ ВЕСОВ ---
SCALE = 128
def to_fixed(weights):
    return [int(round(w * SCALE)) for w in weights.flatten()]

print("\n--- КОД ДЛЯ ВСТАВКИ В VERILOG (Файл weights.txt) ---")

# Layer 1
w1, b1 = model.get_layer('layer1').get_weights()
print(f"localparam [15:0] L1_W [0:{len(to_fixed(w1))-1}] = '{{ {', '.join(map(str, to_fixed(w1)))} }};")
print(f"localparam [15:0] L1_B [0:{len(to_fixed(b1))-1}] = '{{ {', '.join(map(str, to_fixed(b1)))} }};")

# Layer 2
w2, b2 = model.get_layer('layer2').get_weights()
print(f"localparam [15:0] L2_W [0:{len(to_fixed(w2))-1}] = '{{ {', '.join(map(str, to_fixed(w2)))} }};")
print(f"localparam [15:0] L2_B [0:{len(to_fixed(b2))-1}] = '{{ {', '.join(map(str, to_fixed(b2)))} }};")

# Output
w3, b3 = model.get_layer('output').get_weights()
print(f"localparam [15:0] L3_W [0:{len(to_fixed(w3))-1}] = '{{ {', '.join(map(str, to_fixed(w3)))} }};")
print(f"localparam [15:0] L3_B [0:{len(to_fixed(b3))-1}] = '{{ {', '.join(map(str, to_fixed(b3)))} }};")

Проверка GPU...
✅ GPU активен! (PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'))

Генерация всех состояний (19683)...
Чистые данные готовы: 19371 примеров.
Статистика: Побед X: 4123, Побед O: 4123, Ничьих/Пусто: 11125

Запуск обучения (Epochs=150)...
Epoch 1/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.7362 - loss: 0.4879 - val_accuracy: 0.6806 - val_loss: 0.2782
Epoch 2/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6871 - loss: 0.2552 - val_accuracy: 0.6852 - val_loss: 0.1772
Epoch 3/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7050 - loss: 0.1675 - val_accuracy: 0.7265 - val_loss: 0.1318
Epoch 4/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7238 - loss: 0.1219 - val_accuracy: 0.7430 - val_loss: 0.0985
Epoch 5/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7226 - loss: 0.0870 - val_accuracy: 0.7492 - val_loss: 0.0709
Epoch 6/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7286 - loss: 0.0656

KeyboardInterrupt: 

In [ ]:
import numpy as np
import tensorflow as tf
import itertools
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.utils import class_weight

# --- 1. ПРОВЕРКА GPU ---
gpus = tf.config.list_physical_devices('GPU')
print(f"GPU Status: {'✅ Active' if gpus else '⚠️ Not found'}")

# --- 2. ГЕНЕРАЦИЯ ДАННЫХ ---
def check_winner_class(board):
    grid = np.array(board).reshape(3, 3)
    win_x = False
    win_o = False

    lines = []
    for i in range(3):
        lines.append(grid[i, :])
        lines.append(grid[:, i])
    lines.append(grid.diagonal())
    lines.append(np.fliplr(grid).diagonal())

    for line in lines:
        if np.all(line == 1): win_x = True
        if np.all(line == 2): win_o = True

    # Классы: 0 = Ничья/В процессе, 1 = Победа X, 2 = Победа O
    if win_x and win_o: return -1 # Мусор (оба победили)
    if win_x: return 1
    if win_o: return 2
    return 0

print("\nГенерация данных...")
all_states = list(itertools.product([0, 1, 2], repeat=9))

X_raw = []
Y_raw = [] # Теперь это будет просто число 0, 1 или 2

for state in all_states:
    cls = check_winner_class(state)
    if cls == -1: continue # Пропускаем мусор

    # Вход 18 бит
    layer_x = [1 if cell == 1 else 0 for cell in state]
    layer_o = [1 if cell == 2 else 0 for cell in state]

    X_raw.append(layer_x + layer_o)
    Y_raw.append(cls)

X_data = np.array(X_raw)
Y_data = np.array(Y_raw)

# Перемешиваем
indices = np.arange(len(X_data))
np.random.shuffle(indices)
X_data = X_data[indices]
Y_data = Y_data[indices]

# --- ВЫЧИСЛЕНИЕ ВЕСОВ КЛАССОВ ---
# Это самое важное! Мы считаем, сколько раз встречается каждый класс,
# и говорим сети уделять больше внимания редким победам.
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(Y_data),
    y=Y_data
)
class_weights_dict = dict(enumerate(class_weights))
print(f"Веса классов (важность): {class_weights_dict}")
# Вы увидите, что класс 0 (Ничья) имеет малый вес, а 1 и 2 - большой.

# --- 3. АРХИТЕКТУРА (3 ВЫХОДА) ---
model = Sequential()
# Слой 1
model.add(Dense(64, input_dim=18, activation='relu', name='L1'))
# Слой 2
model.add(Dense(32, activation='relu', name='L2'))
# Выход: 3 нейрона (Softmax выбирает лучшего из 3)
model.add(Dense(3, activation='softmax', name='Output'))

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# --- 4. ОБУЧЕНИЕ ---
print("\nЗапуск обучения...")
# Передаем class_weight, чтобы исправить дисбаланс
history = model.fit(
    X_data, Y_data,
    epochs=50,
    batch_size=64,
    verbose=1,
    class_weight=class_weights_dict, # МАГИЯ ЗДЕСЬ
    validation_split=0.1
)

loss, acc = model.evaluate(X_data, Y_data, verbose=0)
print(f"\n🔥🔥 ИТОГОВАЯ ТОЧНОСТЬ: {acc*100:.2f}% 🔥🔥")

# --- 5. ЭКСПОРТ ВЕСОВ ---
if acc > 0.98:
    SCALE = 128
    def to_fixed(w_arr):
        return [int(round(x * SCALE)) for x in w_arr.flatten()]

    print("\n--- COPY TO weights.txt ---")

    # L1
    w, b = model.get_layer('L1').get_weights()
    print(f"localparam [15:0] L1_W [0:{len(to_fixed(w))-1}] = '{{ {', '.join(map(str, to_fixed(w)))} }};")
    print(f"localparam [15:0] L1_B [0:{len(to_fixed(b))-1}] = '{{ {', '.join(map(str, to_fixed(b)))} }};")

    # L2
    w, b = model.get_layer('L2').get_weights()
    print(f"localparam [15:0] L2_W [0:{len(to_fixed(w))-1}] = '{{ {', '.join(map(str, to_fixed(w)))} }};")
    print(f"localparam [15:0] L2_B [0:{len(to_fixed(b))-1}] = '{{ {', '.join(map(str, to_fixed(b)))} }};")

    # Output (3 нейрона)
    w, b = model.get_layer('Output').get_weights()
    print(f"localparam [15:0] L3_W [0:{len(to_fixed(w))-1}] = '{{ {', '.join(map(str, to_fixed(w)))} }};")
    print(f"localparam [15:0] L3_B [0:{len(to_fixed(b))-1}] = '{{ {', '.join(map(str, to_fixed(b)))} }};")
else:
    print("Точность низкая. Попробуй перезапустить обучение еще раз.")

GPU Status: ✅ Active

Генерация данных...
Веса классов (важность): {0: np.float64(0.5804044943820225), 1: np.float64(1.5660926509822946), 2: np.float64(1.5660926509822946)}

Запуск обучения...
Epoch 1/50


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


273/273 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.5008 - loss: 0.8423 - val_accuracy: 0.7848 - val_loss: 0.4872
Epoch 2/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7944 - loss: 0.3982 - val_accuracy: 0.8555 - val_loss: 0.3386
Epoch 3/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8555 - loss: 0.3022 - val_accuracy: 0.8720 - val_loss: 0.3091
Epoch 4/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8912 - loss: 0.2446 - val_accuracy: 0.9190 - val_loss: 0.2248
Epoch 5/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9187 - loss: 0.1924 - val_accuracy: 0.9231 - val_loss: 0.2010
Epoch 6/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9486 - loss: 0.1391 - val_accuracy: 0.9639 - val_loss: 0.1283
Epoch 7/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9648 - loss: 0.1091 - val_accuracy: 0.9757 - val_loss: 0.0998
Epoch 8/50
273/273 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9744 - loss: 0.0839 - val_accuracy: 0.9819 - val

In [ ]:
print("--- ГЕНЕРАЦИЯ TESTBENCH (3 КЛАССА) ---")
# Этот код генерирует проверку для Verilog

# Выберем 5 случайных примеров + специфичные
test_indices = [0, 1, 10, 50, 100]

print("// Вставьте этот код в initial block вашего Testbench.sv")
print("initial begin")
print("    // Reset sequence")
print("    rst = 1; start = 0; #20; rst = 0; #20;")

for i in test_indices:
    in_vec = X_data[i]
    y_true = Y_data[i] # 0, 1 или 2

    # Формируем строку битов для Verilog (18 бит)
    # X_data хранит int 0/1, склеиваем в строку
    bin_str = "".join(map(str, map(int, in_vec)))

    print(f"\n    // Test Case {i}: Ожидаем класс {y_true} (0=Draw, 1=X, 2=O)")

    # Разделяем на X и O для удобства чтения (как мы планировали с буферизацией)
    # Первые 9 бит - крестики, вторые 9 бит - нолики
    print(f"    // Loading X inputs")
    print(f"    switches = 9'b{bin_str[0:9]};")
    print(f"    btn_load_x = 1; #10; btn_load_x = 0; #10;")

    print(f"    // Loading O inputs")
    print(f"    switches = 9'b{bin_str[9:18]};")
    print(f"    btn_load_o = 1; #10; btn_load_o = 0; #10;")

    print(f"    start = 1; #100; start = 0; // Запуск вычисления")

    # Проверка результата
    # В Verilog у нас будет 3 выхода: win_x, win_o, draw
    if y_true == 1:
        print(f"    if (led_win_x !== 1) $display(\"ERROR Case {i}: Expected X Win\");")
        print(f"    else $display(\"PASS Case {i}: X Win Correct\");")
    elif y_true == 2:
        print(f"    if (led_win_o !== 1) $display(\"ERROR Case {i}: Expected O Win\");")
        print(f"    else $display(\"PASS Case {i}: O Win Correct\");")
    else:
        print(f"    if (led_draw !== 1) $display(\"ERROR Case {i}: Expected Draw\");")
        print(f"    else $display(\"PASS Case {i}: Draw Correct\");")

print("\n    $finish;")
print("end")

--- ГЕНЕРАЦИЯ TESTBENCH (3 КЛАССА) ---
// Вставьте этот код в initial block вашего Testbench.sv
initial begin
    // Reset sequence
    rst = 1; start = 0; #20; rst = 0; #20;

    // Test Case 0: Ожидаем класс 2 (0=Draw, 1=X, 2=O)
    // Loading X inputs
    switches = 9'b010000001;
    btn_load_x = 1; #10; btn_load_x = 0; #10;
    // Loading O inputs
    switches = 9'b101111010;
    btn_load_o = 1; #10; btn_load_o = 0; #10;
    start = 1; #100; start = 0; // Запуск вычисления
    if (led_win_o !== 1) $display("ERROR Case 0: Expected O Win");
    else $display("PASS Case 0: O Win Correct");

    // Test Case 1: Ожидаем класс 1 (0=Draw, 1=X, 2=O)
    // Loading X inputs
    switches = 9'b000001111;
    btn_load_x = 1; #10; btn_load_x = 0; #10;
    // Loading O inputs
    switches = 9'b100010000;
    btn_load_o = 1; #10; btn_load_o = 0; #10;
    start = 1; #100; start = 0; // Запуск вычисления
    if (led_win_x !== 1) $display("ERROR Case 1: Expected X Win");
    else $display("PASS Case